In [37]:
import findspark
findspark.init()
findspark.find()

'H:\\SPARK'

In [38]:
from pyspark.sql import SparkSession

# Initialize SparkSession with necessary configurations
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Spark') \
    .config("spark.driver.memory", "15g") \
    .config("spark.hadoop.home.dir", "H:/HADOOP/") \
    .config("spark.hadoop.conf.dir", "H:/HADOOP/etc/hadoop/") \
    .getOrCreate()
    
import sys
sys.path.append("G:\Dissertation_Project")

# Get SparkContext from the SparkSession
sc = spark.sparkContext


In [39]:
spark

### BASE DATASET

In [40]:
base_df = spark.read.csv("../../Data/Custom_Datasets/conversation_datasets_GPT.csv", header=True, inferSchema=True)
base_df.show(10, truncate=False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|Conversation_ID|Attacker_Helper                                                                                                                                                 |Victim                                                                                                                                                                                         |Conversation_Type|
+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------

### PREPROCESSED DATASET

In [41]:
preprocessed_df = spark.read.csv("../../Data/Preprocessed_Datasets/GPT_dataset_preprocessed.csv", header=True, inferSchema=True)
preprocessed_df.show(10, truncate=False)

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Convert Conversation Columns into actual Arrays

In [42]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import ast

# UDF to convert string representation of list to actual list
def str_to_array_of_arrays(s):
    # Convert the string to a list and then wrap it inside another list
    return [ast.literal_eval(s)][0]

str_to_array_of_arrays_udf = udf(str_to_array_of_arrays, ArrayType(ArrayType(StringType())))

df = preprocessed_df.withColumn("Attacker_Helper", str_to_array_of_arrays_udf(preprocessed_df["Attacker_Helper"])).withColumn("Victim", str_to_array_of_arrays_udf(preprocessed_df["Victim"]))

df.printSchema()

root
 |-- Conversation_ID: string (nullable = true)
 |-- Attacker_Helper: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Victim: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Conversation_Type: integer (nullable = true)



### Loading the pipeline and transforming the data

In [43]:
from pyspark.ml import PipelineModel
from src.CustonTransformers import FlattenTransformer

pipeline_model_path = "./Pipelines/TF-IDF_Pipeline"

pipeline = PipelineModel.load(path=pipeline_model_path)

df_assembled = pipeline.transform(df)

### Splitting the Data

In [44]:
(train_data, test_data) = df_assembled.randomSplit([0.8, 0.2], seed=42)
train_data.show(truncate=False)

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# LOGISTIC REGRESSION

### Train the Logistic Regression Model

In [45]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(featuresCol="combined_features", labelCol="Conversation_Type", predictionCol="Prediction")

evaluator = MulticlassClassificationEvaluator(labelCol='Conversation_Type', metricName="weightedRecall", predictionCol="Prediction")


paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [10, 50, 100]) \
    .addGrid(lr.regParam, [0.01, 0.1, 1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.4, 0.6, 1.0]) \
    .build()


# Cross Validate with weightedRecall as a metric
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=7) # high number of num folds because the dataset is small


cvModel = crossval.fit(train_data)

print(cvModel, '\n')
print(cvModel.explainParams(), '\n')


CrossValidatorModel_83eb870e3651 

estimator: estimator to be cross-validated (current: LogisticRegression_26c19b40a256)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LogisticRegression_26c19b40a256', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_26c19b40a256', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_26c19b40a256', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, {Param(parent='LogisticRegression_26c19b40a256', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_26c19b40a256', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_26c19b40a256', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 pen

In [46]:
# Print the best model's parameters
bestModelParams = cvModel.bestModel.extractParamMap()
for param, value in bestModelParams.items():
    print(f"{param.name}: {value}")

aggregationDepth: 2
elasticNetParam: 0.0
family: auto
featuresCol: combined_features
fitIntercept: True
labelCol: Conversation_Type
maxBlockSizeInMB: 0.0
maxIter: 50
predictionCol: Prediction
probabilityCol: probability
rawPredictionCol: rawPrediction
regParam: 0.01
standardization: True
threshold: 0.5
tol: 1e-06


### Evaluating The Models Performance on the Test set across various metrics

In [47]:
# The model is trained with weightedRecall in mind 
metrics = ['accuracy', 'f1', 'weightedPrecision', 'weightedRecall']
for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(labelCol='Conversation_Type', metricName=metric, predictionCol="Prediction")   
    score = evaluator.evaluate(cvModel.transform(test_data))
    print(f"Metric: {metric}---\t\t----Score: {score}")

Metric: accuracy---		----Score: 0.9456521739130435
Metric: f1---		----Score: 0.9457238299225073
Metric: weightedPrecision---		----Score: 0.9460212504938349
Metric: weightedRecall---		----Score: 0.9456521739130435


In [48]:
print('Best Model -> {}'.format(cvModel.bestModel))

Best Model -> LogisticRegressionModel: uid=LogisticRegression_26c19b40a256, numClasses=2, numFeatures=400


### Testing

In [49]:
predictions = cvModel.bestModel.transform(test_data)
predictions.select("Conversation_ID","Conversation_Type", "Prediction", "Probability").where(predictions.Prediction != predictions.Conversation_Type).show(10, truncate=False)

+---------------+-----------------+----------+----------------------------------------+
|Conversation_ID|Conversation_Type|Prediction|Probability                             |
+---------------+-----------------+----------+----------------------------------------+
|4bMquyU0KyOnOH |1                |0.0       |[0.8584422981449245,0.14155770185507555]|
|DmgYMeupfuuUtH |1                |0.0       |[0.8113005963069706,0.1886994036930294] |
|VJchs3LM0RvvnW |0                |1.0       |[0.49452232522102335,0.5054776747789766]|
|sT9TcijfaCXEoI |1                |0.0       |[0.7697166253539466,0.23028337464605342]|
|wNsWdbpeld     |0                |1.0       |[0.4073522019366856,0.5926477980633145] |
+---------------+-----------------+----------+----------------------------------------+



In [50]:
model_path = "../Models/Trained_Models/LogisticRegression"
print(type(model_path))

<class 'str'>


In [51]:
cvModel.write().overwrite().save(model_path)